In [23]:
from langchain_groq import ChatGroq
import key

In [24]:
llm=ChatGroq(
    model_name="llama3-8b-8192",
    temperature=0.7,
    max_tokens=None,
    timeout=None,
    max_retries=3,
    api_key=key.Dishly_key
    )

In [25]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://careers.nike.com/analyst-finance-gc/job/R-45280")
page_data = loader.load().pop().page_content
print(page_data)






















Analyst, Finance GC










































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  עִברִית  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers










Language











Menu



Return to Previous Menu



Select

In [26]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [27]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Analyst, Finance GC',
 'experience': 'At least 5 years’ experience of audit or finance analyst position in MNC, Media or Creative agency working experience is a plus',
 'skills': 'Proficiency with office software such as Microsoft Excel, Outlook, Word and PowerPoint, Power BI and tableau skill is preferred',
 'description': 'Key Responsibilities: This is a commercial FP&A role and key responsibility is to support Icon. Shanghai Studio Financial planning and analysis for Business team. Support monthly actual and forecast spending collection, validation, and variance analysis. Support system maintenance and development to facilitate Icon financial process. Support annual budgeting and 3 years strategic plan development. Prepare financial related review reports and deliverables. Cross function communication with Icon. Studio team and other finance team. Other ad hoc tasks.'}

In [28]:
type(json_res)

dict

In [29]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular, .NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [30]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [31]:
job = json_res[0]
jobs=job['skills']

KeyError: 0

In [32]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/xamarin-portfolio'}]]

In [33]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Harshil Amin, a business development executive at Karma. Karma is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of Karma 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Harshil Amin, BDE at Karma. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Enhancing Financial Planning and Analysis for Icon Shanghai Studio

Dear [Client],

I came across an exciting job posting for an Analyst, Finance GC at Icon Shanghai Studio, and I immediately thought of Karma's expertise in supporting businesses like yours with tailored solutions. As a business development executive at Karma, I'd like to introduce our capabilities in finance planning and analysis, which align perfectly with your job requirements.

At Karma, we specialize in facilitating the seamless integration of business processes through automated tools. Our expertise in financial planning and analysis can support Icon Shanghai Studio's financial planning and analysis, including:

* Financial planning and budgeting: Our team can help streamline financial planning and budgeting processes, ensuring accurate forecasting and variance analysis.
* System development and maintenance: We can assist in developing and maintaining financial systems to facilitate Icon's financial proce